In [ ]:
## Creating a basic Naives Base News Classifier for document classification

In [4]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.book import * 


In [5]:
#create dataset of articles to build classification model with 
#Ref: http://scikit-learn.org/stable/datasets/twenty_newsgroups.html
from sklearn.datasets import fetch_20newsgroups 
from pprint import pprint

#fetch articles for training 
#note:the sklearn.datasets.fetch_20newsgroups function is a data fetching / caching functions that
    #downloads the data archive from the original 20 newsgroups website, extracts the archive contents in
    #the ~/scikit_learn_data/20news_home folder and calls the sklearn.datasets.load_files on either the 
    #training or testing set folder, or both of them:
newsgroups_train = fetch_20newsgroups(subset='train')
pprint(list(newsgroups_train.target_names))


#look at data properties 
#note:The real data lies in the filenames and target attributes. The target attribute is the integer index of the category
newsgroups_train.filenames.shape #total files or groups of articles

#different datasets that can be called from the newsgroups list of articles
list(newsgroups_train)
type(newsgroups_train.data)

#all childs associated with parent newsgroup_train
print(dir(newsgroups_train))

#location of files being used for dataset
print(newsgroups_train.filenames)

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']
['DESCR', 'data', 'description', 'filenames', 'target', 'target_names']
[ 'C:\\Users\\caridza\\scikit_learn_data\\20news_home\\20news-bydate-train\\rec.autos\\102994'
 'C:\\Users\\caridza\\scikit_learn_data\\20news_home\\20news-bydate-train\\comp.sys.mac.hardware\\51861'
 'C:\\Users\\caridza\\scikit_learn_data\\20news_home\\20news-bydate-train\\comp.sys.mac.hardware\\51879'
 ...,
 'C:\\Users\\caridza\\scikit_learn_data\\20news_home\\20news-bydate-train\\comp.sys.ibm.pc.hardware\\60695'
 'C:\\Users\\caridza\\scikit_learn_data\\20news_home\\20news-bydate-train\\comp.graphics\\38319'

In [15]:
list(newsgroups_train)


['target', 'target_names', 'data', 'filenames', 'description', 'DESCR']

In [4]:
#subset of categories to include in model
categories = ['alt.atheism', 'talk.religion.misc','comp.graphics', 'sci.space']

#elements of text to remove from original articles before processing 
remove =('headers','footers','quotes')

#training data
newsgroups_train = fetch_20newsgroups(subset='train',remove=remove,categories=categories)
train_target = [newsgroups_train.target,newsgroups_train.target_names]

#test data
newsgroups_test = fetch_20newsgroups(subset = 'test', remove =remove, categories=categories)
test_target = [newsgroups_test.target,newsgroups_test.target_names]

#tfidf vectorize 
tfidf_vectorizer = TfidfVectorizer(stop_words = "english",strip_accents ='unicode', analyzer ='word', smooth_idf=False)

#create vector of training data 
vectors_train = tfidf_vectorizer.fit_transform(newsgroups_train.data)

#create vector of test data
vectors_test = tfidf_vectorizer.transform(newsgroups_test.data)

In [16]:
print('Total Features in tdif:',vectors_train.shape)

#sparcity of vector (avg 96 non-zero components by sample in more than 26K dimensional space)
print("Avg Non Zero Components over "+str(vectors_train.shape[1])+" total dimensions: ",str(vectors_train.nnz / float(vectors_train.shape[0])))

Total Features in tdif: (2034, 26576)
Avg Non Zero Components over 26576 total dimensions:  65.70009832841691


### Training and Scoring MultiNomial Naive Bayes Classifier

In [19]:
#Create basic mulinomial Naive Bayes classifer 
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

#Naive Bayes Model Characteristics(Given a particular piece of data, how likely is a particular outcome?)
    #1.Expects integer inputs,and can be used for multiple label classification
    #2.May not work well with tf-idf inputs, for this situation use svm models
    #3.Basis in probability 
    #4.Each term in count_vectorization is a feature 

#create model object 
clf = MultinomialNB(alpha=.01)

#fit model on training data 
#determines internal parameters based on training dataset
#pass training count vectorizor first, and training labels second
clf.fit(vectors_train, train_target[0])

#make predictions on test data 
pred = clf.predict(vectors_test)

#model validation metrics (the percentage of correct genre guesses out of total guesses)
print(metrics.f1_score(test_target[0], pred, average = 'macro'))
print(metrics.confusion_matrix(test_target[0], pred, labels=[0,1]))

#identify top 10 model features for each target level
show_top10(clf, tfidf_vectorizer, train_target[1])

0.766718790018
[[209   9]
 [ 10 346]]
alt.atheism: islam does religion atheism say just think don people god
comp.graphics: windows does looking program know file image files thanks graphics
sci.space: earth think shuttle orbit moon just launch like nasa space
talk.religion.misc: objective think just bible don christians christian people jesus god


### Misc: Map the important vector weights back to actual words using some simple inspection techniques.     ####### Note: for this exercise removing alphanumerics makes alot of sence 


In [38]:
# Target class labels of Naives base model(1 for each article type) 
class_labels = clf.classes_

# Extract features from tfidf vectors(all possible independent vars to consider)
feature_names = tfidf_vectorizer.get_feature_names()

#Create a zipped array of the classifier coefficients with the feature names and sort them by the coefficients. 
#first use zip() with the arguments nb_classifier.coef_[0] and feature_names. Then, use sorted() on this.
feat_with_weights = sorted(zip(clf.coef_[0], feature_names))

# Print the first class label and the top 20 feat_with_weights entries
print(class_labels[0], feat_with_weights[:20])

# Print the second class label and the bottom 20 feat_with_weights entries
print(class_labels[1], feat_with_weights[-20:])

(0, [(-12.602384314982151, u'00'), (-12.602384314982151, u'0000'), (-12.602384314982151, u'00000'), (-12.602384314982151, u'000000'), (-12.602384314982151, u'000005102000'), (-12.602384314982151, u'000062david42'), (-12.602384314982151, u'0001'), (-12.602384314982151, u'000100255pixel'), (-12.602384314982151, u'00041032'), (-12.602384314982151, u'0004136'), (-12.602384314982151, u'0004246'), (-12.602384314982151, u'0004422'), (-12.602384314982151, u'00044513'), (-12.602384314982151, u'0004847546'), (-12.602384314982151, u'0005'), (-12.602384314982151, u'0007'), (-12.602384314982151, u'00090711'), (-12.602384314982151, u'000usd'), (-12.602384314982151, u'0012'), (-12.602384314982151, u'001200201pixel')])
(1, [(-6.3827111108702921, u'moral'), (-6.3779125797818521, u'bobby'), (-6.372219081995393, u'time'), (-6.3356459842503785, u'objective'), (-6.28236240974321, u'bible'), (-6.2690085842012522, u'like'), (-6.2258362809188856, u'believe'), (-6.1962344777436922, u'said'), (-6.16811001327871

[(-10.794670308847721, u'00'),
 (-8.5717545847047738, u'000'),
 (-12.432877241179266, u'0000'),
 (-12.432877241179266, u'00000'),
 (-12.432877241179266, u'000000'),
 (-12.432877241179266, u'000005102000'),
 (-12.432877241179266, u'000062david42'),
 (-9.9771848425229717, u'0001'),
 (-12.432877241179266, u'000100255pixel'),
 (-12.432877241179266, u'00041032'),
 (-12.432877241179266, u'0004136'),
 (-12.432877241179266, u'0004246'),
 (-12.432877241179266, u'0004422'),
 (-12.432877241179266, u'00044513'),
 (-12.432877241179266, u'0004847546'),
 (-12.432877241179266, u'0005'),
 (-12.432877241179266, u'0007'),
 (-12.432877241179266, u'00090711'),
 (-12.432877241179266, u'000usd'),
 (-12.432877241179266, u'0012'),
 (-12.432877241179266, u'001200201pixel'),
 (-12.432877241179266, u'0018'),
 (-12.432877241179266, u'00196'),
 (-12.432877241179266, u'0020'),
 (-12.432877241179266, u'0022'),
 (-12.432877241179266, u'0028'),
 (-12.432877241179266, u'0029'),
 (-12.432877241179266, u'0033'),
 (-12.432

## Imporve original model by iterating through sereies of alphas 
#### NOTE: Function must be compiled first to run


In [20]:
#create range of alphas to loop model through 
alphas = np.arange(0,1,.1)


#define train and predict 
#target series must be vector (1 dimensional) series 
for alpha in alphas:
    print(NaivesBase_TrainAndPredict(train_data=vectors_train
                           , train_target= train_target[0]
                           , test_data=vectors_test
                           , test_target=test_target[0]
                           , alpha=alpha
                          ))

    
    

0.751662971175
0.791574279379
0.784922394678
0.778270509978
0.764966740576
0.759053954176
0.753141167775
0.752402069475
0.748706577975
0.745011086475


In [188]:
#COUNT VECTORIZE
#using pandas series to process data and sklearn to generate Tfidf
from sklearn.feature_extraction.text import TfidfVectorizer

#Independent Data Series: train and test
train_X = pd.Series(newsgroups_train.data)
test_X = pd.Series(newsgroups_test.data)

#Dependent Data Series 
train_Y = pd.Series(newsgroups_train.target)
test_Y = pd.Series(newsgroups_test.target)

#################################################
##############COUNT VECTORIZATION################
#################################################
# Initialize a CountVectorizer object: count_vectorizer
count_vectorizer = CountVectorizer(stop_words='english')

# Transform the training data using only the 'text' column values: count_train 
count_train = count_vectorizer.fit_transform(train_X)
count_test = count_vectorizer.transform(test_X)

#################################################
###############TDIF VECTORIZATION################
#################################################
## Initialize a TfidfVectorizer object
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

# Transform the training data: tfidf_train 
tfidf_train = tfidf_vectorizer.fit_transform(train_X)

# Transform the test data: tfidf_test 
tfidf_test = tfidf_vectorizer.transform(test_X)

# Create a Multinomial Naive Bayes classifier: nb_classifier
nb_classifier = MultinomialNB()

# Fit the classifier to the training data
nb_classifier.fit(tfidf_train, train_Y)

# Create the predicted tags: pred
pred = nb_classifier.predict(tfidf_test)

# Calculate the accuracy score: score
score = metrics.accuracy_score(test_Y, pred)
print(score)

# Calculate the confusion matrix: cm
cm = metrics.confusion_matrix(test_Y, pred)
print(cm)



# Create the CountVectorizer DataFrame: count_df
#specifying the values as the first argument and the columns (or features) as the second argument
#Note:The values can be accessed by using the .A attribute of, respectively, count_train and tfidf_train
#Note:The columns can be accessed using the .get_feature_names() methods of count_vectorizer and tfidf_vectorizer
count_df = pd.DataFrame(count_train.A, columns=count_vectorizer.get_feature_names())

# Create the TfidfVectorizer DataFrame: tfidf_df
tfidf_df = pd.DataFrame(tfidf_train.A, columns=tfidf_vectorizer.get_feature_names())



# Print the head of tfidf_df
print(tfidf_df.head())

# Calculate the difference in columns: difference
difference = set(count_df.columns) - set(tfidf_df.columns)
print(difference)

# Check whether the DataFrames are equal
print(count_df.equals(tfidf_df))

#metadata for objects above
print('Count Vectorization Summary')
print('newsgroups_test is:',type(newsgroups_test))
print('count_vectorizer is:',type(count_vectorizer))
print('count_train is:',type(count_train))
print('Objects callable within newsgroup_test object:','\n',dir(newsgroups_test))
print('CountVectorizer Settable Parameters: ','\n',count_vectorizer.get_params)
print('\n')
print('TFIDF Vectorization Summary')
print('First 10 features of tfidf vector: ',tfidf_vectorizer.get_feature_names()[:10])
print('First 5 Vectors of the tfidf Training Data:',tfidf_train.A[:5])


0.742054693274
[[225  14  64  16]
 [  4 358  27   0]
 [ 10  21 363   0]
 [135  14  44  58]]
    00  000  0000  00000  000000  000005102000  000062david42  0001  \
0  0.0  0.0   0.0    0.0     0.0           0.0            0.0   0.0   
1  0.0  0.0   0.0    0.0     0.0           0.0            0.0   0.0   
2  0.0  0.0   0.0    0.0     0.0           0.0            0.0   0.0   
3  0.0  0.0   0.0    0.0     0.0           0.0            0.0   0.0   
4  0.0  0.0   0.0    0.0     0.0           0.0            0.0   0.0   

   000100255pixel  00041032  ...    zurich  zurvanism  zus  zvi  \
0             0.0       0.0  ...       0.0        0.0  0.0  0.0   
1             0.0       0.0  ...       0.0        0.0  0.0  0.0   
2             0.0       0.0  ...       0.0        0.0  0.0  0.0   
3             0.0       0.0  ...       0.0        0.0  0.0  0.0   
4             0.0       0.0  ...       0.0        0.0  0.0  0.0   

   zwaartepunten  zwak  zwakke  zware  zwarte  zyxel  
0            0.0   0.0 

In [24]:
##FUNCTIONS 

#most informative vectorized features 
def show_top10(classifier, vectorizer, categories):
    feature_names = np.asarray(vectorizer.get_feature_names())
    for i, category in enumerate(categories):
         top10 = np.argsort(classifier.coef_[i])[-10:]
         print("%s: %s" % (category, " ".join(feature_names[top10])))

        
def NaivesBase_TrainAndPredict(train_data,train_target,test_data,test_target, alpha):
    clf = MultinomialNB(alpha=alpha)
    clf.fit(train_data,train_target)
    pred = clf.predict(test_data)
    score = metrics.accuracy_score(test_target,pred)
    return(score)
    
#get files
def getfiles(path, extension):
    items = os.listdir(path)
    newlist = []
    for names in items:
        if names.endswith(extension):
            newlist.append(names)
    return newlist

#determine if a class of variable has a specific attribute associated with it(an option you can change/specifcy) 
all(hasattr(cls, '__len__') for cls in (str, bytes, tuple, list, dict, set, frozenset))


True

In [ ]:

#use webscraping to scrape articles of various topics and automate initial document type classification based on article type
#goal: eliminate requirement of manual labeling data for training document classifcaiton models 
#http://localhost:57758/?token=3e34c9838ca8b9c122ed89efa3db22eafcc4bee222f712f1

    #How to create an NLTK corpus with a directory of textfiles?
    #https://stackoverflow.com/questions/4951751/creating-a-new-corpus-with-nltk
